# Degree Day Analysis


In [13]:
from pathlib import Path
import datetime as dt
import numpy as np
import pandas as pd
import hvplot.pandas
import sqlalchemy
import helpful_functions as hf
%matplotlib inline

## Import Data from Database

In [14]:
engine = sqlalchemy.create_engine(hf.db_connection_string)
inspector = sqlalchemy.inspect(engine)
# print(engine)
# inspector.get_table_names()

Engine(sqlite:///Resources/energy_data.db)


['DegreeDays_2020',
 'DegreeDays_2021',
 'DegreeDays_Austin_2020',
 'DegreeDays_Austin_2021',
 'DegreeDays_Corpus_Christi_2020',
 'DegreeDays_Corpus_Christi_2021',
 'DegreeDays_Dallas_2020',
 'DegreeDays_Dallas_2021',
 'DegreeDays_Houston_2020',
 'DegreeDays_Houston_2021',
 'DegreeDays_San_Angelo_2020',
 'DegreeDays_San_Angelo_2021',
 'DegreeDays_San_Antonio_2020',
 'DegreeDays_San_Antonio_2021',
 'ERCOT_2020',
 'ERCOT_2021',
 'ERCOT_Austin_2020',
 'ERCOT_Austin_2021',
 'ERCOT_Corpus_Christi_2020',
 'ERCOT_Corpus_Christi_2021',
 'ERCOT_Dallas_2020',
 'ERCOT_Dallas_2021',
 'ERCOT_Houston_2020',
 'ERCOT_Houston_2021',
 'ERCOT_San_Angelo_2020',
 'ERCOT_San_Angelo_2021',
 'ERCOT_San_Antonio_2020',
 'ERCOT_San_Antonio_2021',
 'WU_Austin_2020',
 'WU_Austin_2021',
 'WU_Corpus_Christi_2020',
 'WU_Corpus_Christi_2021',
 'WU_Dallas_2020',
 'WU_Dallas_2021',
 'WU_Houston_2020',
 'WU_Houston_2021',
 'WU_San_Angelo_2020',
 'WU_San_Angelo_2021',
 'WU_San_Antonio_2020',
 'WU_San_Antonio_2021']

In [15]:
def gen_dd_df(
    city, 
    year, 
    data_source='DegreeDays',
    new_col_name=None,
    old_col_name='HDD 65', 
    engine=engine
):
    df = pd.read_sql_table(
        '_'.join([data_source, city, year]),
        con=engine
    )
    if new_col_name is not None:
        df.rename(columns={old_col_name: new_col_name}, inplace=True)
    return df



In [16]:
dti_dict = {
    '2020' : pd.DataFrame(pd.date_range("2020-02-01", periods=24*28*4, freq="15T", name='Date')),
    '2021' : pd.DataFrame(pd.date_range("2021-02-01", periods=24*28*4, freq="15T", name='Date'))
}

dd_df_dict = {}
for city in hf.city_list_2:
    dd_df_dict[city] = {}
    for year in hf.year_list:
        print('\n', 80*'_', city, year, )
        dd_df_dict[city][year] = gen_dd_df(city, year)
        dd_df_dict[city][year] = pd.concat([dd_df_dict[city][year], dti_dict[year]],join='outer',)
        dd_df_dict[city][year].sort_values('Date', inplace=True)
        dd_df_dict[city][year] = dd_df_dict[city][year].reindex(columns=['City', 'Year', 'Date', 'Overlay_Date', 'HDD 65']).reset_index(drop=True)
        dd_df_dict[city][year]['HDD 65'] = dd_df_dict[city][year]['HDD 65'].interpolate(method='pad')
        dd_df_dict[city][year]['City'] = city
        dd_df_dict[city][year]['Year'] = year
        dd_df_dict[city][year]['Overlay_Date'] = dd_df_dict[city][year]['Date'].apply(lambda x: x.replace(year=1900))
        dd_df_dict[city][year]['Date'] = dd_df_dict[city][year]['Date'].apply(lambda x: x.to_pydatetime())
        print(dd_df_dict[city][year].head())


 ________________________________________________________________________________ Austin 2020
     City  Year                Date        Overlay_Date  HDD 65
0  Austin  2020 2020-02-01 00:00:00 1900-02-01 00:00:00    15.2
1  Austin  2020 2020-02-01 00:00:00 1900-02-01 00:00:00    15.2
2  Austin  2020 2020-02-01 00:15:00 1900-02-01 00:15:00    15.2
3  Austin  2020 2020-02-01 00:30:00 1900-02-01 00:30:00    15.2
4  Austin  2020 2020-02-01 00:45:00 1900-02-01 00:45:00    15.2

 ________________________________________________________________________________ Austin 2021
     City  Year                Date        Overlay_Date  HDD 65
0  Austin  2021 2021-02-01 00:00:00 1900-02-01 00:00:00    15.4
1  Austin  2021 2021-02-01 00:00:00 1900-02-01 00:00:00    15.4
2  Austin  2021 2021-02-01 00:15:00 1900-02-01 00:15:00    15.4
3  Austin  2021 2021-02-01 00:30:00 1900-02-01 00:30:00    15.4
4  Austin  2021 2021-02-01 00:45:00 1900-02-01 00:45:00    15.4

 ________________________________________